In [1]:
from protobackend.dispatcher import Dispatcher

Background
----------

At it's core, the `Dispatcher` object calls methods belonging to an `active_exercise`.
In the examples below, the "exercises" are just the `math` and `os.path` modules.

The dispatcher works as follows

1. Call any defined pre_hooks
2. Dispatch command
3. Call any defined post_hooks

Dispatching Commands
--------------------

Commands are dispatched to the `active_exercise` using the `Dispatcher.dispatch` method. This method expects a dictionary with two entries:

1. **command**: the name of the method being called
2. **payload**: passed as the first argument when that method is called

The example below shows a simple dispatcher, which calls `math.ceil(2.5)`.

In [2]:
import math

d = Dispatcher({})
d.active_exercise = math
d.dispatch({'command': 'ceil', 'payload': 2.5})

3

Defining pre/post hooks
-----------------------

`Dispatcher` instances have a `hook` decorator, that can be used to define behavior before and after dispatching each command. In the example below, pre hooks convert string payloads to float, and add 1 to it when `ceil` is being called.

In [3]:
import math

d = Dispatcher({})
d.active_exercise = math

@d.hook('pre')
def str_to_float(data, cmd):
    return float(data)

@d.hook('pre')
def add1_to_ceil(data, cmd):
    return data + 1 if cmd == 'ceil' else data

@d.hook('post')
def print_output(data, cmd):
    print('%s output:\t'%cmd, data)
    return data

d.dispatch({'command': 'ceil', 'payload': '2.5'})
d.dispatch({'command': 'floor', 'payload': '2.5'})

ceil output:	 4
floor output:	 2


2

Switching exercises
-------------------

Hooks may take an optional third argument named dispatcher.
This allows you to set the active exercise within a pre hook.
The example demonstrates this by switching between the `math` and `path` modules, depending on the command.

In [4]:
import math
from os import path

d = Dispatcher({'math': math, 'path': path})

@d.hook('pre')
def set_active(data, cmd, dispatcher):
    # note that the modules may also be accessed using
    # dispatcher.exercises
    if cmd == 'split':
        dispatcher.active_exercise = path
    else:
        dispatcher.active_exercise = math
    
    return data

@d.hook('post')
def print_output(data, cmd):
    print('%s:\t'%cmd, data)

d.dispatch({'command': 'split', 'payload': 'a/b/c.txt'})
d.dispatch({'command': 'ceil', 'payload': 2.5})

split:	 ('a/b', 'c.txt')
ceil:	 3
